In [ ]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/train_tree_ready.csv")
test  = pd.read_csv("/content/drive/MyDrive/test_tree_ready.csv")

print(train.shape, test.shape)
train.head()

(70000, 31) (30000, 30)


,seller_address,warranty,condition,shipping,non_mercado_pago_payment_methods,seller_id,listing_type_id,price,attributes,tags,...,has_sales,listing_group,category_freq,ship_local_pick_up,ship_free_shipping,ship_mode,ship_has_methods,addr_country,addr_state,addr_city
0,"{'country': {'name': 'Argentina', 'id': 'AR'},...",Calidad y mejor precio,new,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",4086929269,bronze,15.02,NaN,['dragged_bids_and_visits'],...,0,medium,0.001586,1,0,not_specified,0,Argentina,Buenos Aires,Quilmes
1,"{'country': {'name': 'Argentina', 'id': 'AR'},...",6 meses,new,"{'local_pick_up': True, 'methods': [], 'tags':...",NaN,7222239700,bronze,5520.00,NaN,NaN,...,0,medium,0.000043,1,0,not_specified,0,Argentina,Córdoba,Córdoba
2,"{'country': {'name': 'Argentina', 'id': 'AR'},...",NaN,used,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",1012912452,free,100.00,"[{'value_id': 'female', 'attribute_group_id': ...",['dragged_bids_and_visits'],...,0,low,0.000643,1,0,not_specified,0,Argentina,Buenos Aires,Jose Marmol
3,"{'country': {'name': 'Argentina', 'id': 'AR'},...",Dada por la trayectoria de la marca,used,"{'local_pick_up': False, 'methods': [], 'tags'...","[{'description': 'Transferencia bancaria', 'id...",4449250674,free,65.00,"[{'value_id': 'boys', 'attribute_group_id': 'D...",['dragged_bids_and_visits'],...,0,low,0.000157,0,0,custom,0,Argentina,Capital Federal,Villa Crespo
4,"{'country': {'name': 'Argentina', 'id': 'AR'},...",NaN,new,"{'local_pick_up': True, 'methods': [], 'tags':...",NaN,5964151586,bronze,45.00,NaN,['dragged_bids_and_visits'],...,1,medium,0.007743,1,0,not_specified,0,Argentina,Capital Federal,caba


In [ ]:
y = train["condition"].map({"new":1, "used":0})
X = train.drop(columns=["condition"])

In [ ]:
cat_features = [col for col in X.columns if X[col].dtype == "object"]
cat_features[:10], len(cat_features)

(['seller_address',
  'warranty',
  'shipping',
  'non_mercado_pago_payment_methods',
  'listing_type_id',
  'attributes',
  'tags',
  'category_id',
  'descriptions',
  'international_delivery_mode'],
 18)

In [ ]:
for col in cat_features:
    X[col] = X[col].astype(str).fillna("NA")
    test[col] = test[col].astype(str).fillna("NA")

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=1200,
    depth=8,
    learning_rate=0.03,
    loss_function="Logloss",
    eval_metric="Accuracy",
    random_seed=42,
    verbose=200
)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    cat_features=cat_features
)

0:	learn: 0.8303393	test: 0.8324286	best: 0.8324286 (0)	total: 424ms	remaining: 8m 28s
200:	learn: 0.8940893	test: 0.9031429	best: 0.9031429 (200)	total: 1m 17s	remaining: 6m 25s
400:	learn: 0.9000714	test: 0.9041429	best: 0.9045000 (382)	total: 2m 35s	remaining: 5m 8s
600:	learn: 0.9063393	test: 0.9057857	best: 0.9058571 (593)	total: 3m 53s	remaining: 3m 52s
800:	learn: 0.9116607	test: 0.9065000	best: 0.9069286 (670)	total: 5m 19s	remaining: 2m 39s
1000:	learn: 0.9158036	test: 0.9073571	best: 0.9073571 (1000)	total: 6m 41s	remaining: 1m 19s
1199:	learn: 0.9198036	test: 0.9085714	best: 0.9085714 (1199)	total: 8m 1s	remaining: 0us

bestTest = 0.9085714286
bestIteration = 1199



In [ ]:
model.fit(X, y, cat_features=cat_features, verbose=200)


pred_test = model.predict(test)

sub = pd.DataFrame({
    "ID": range(1, len(test)+1),
    "condition": pd.Series(pred_test.astype(int)).map({1:"new", 0:"used"})
})

sub.to_csv("catboost_treated.csv", index=False)
print("guardado")


0:	learn: 0.8600857	total: 388ms	remaining: 7m 44s
200:	learn: 0.8958857	total: 1m 24s	remaining: 6m 59s
400:	learn: 0.9019714	total: 2m 49s	remaining: 5m 37s
600:	learn: 0.9074571	total: 4m 18s	remaining: 4m 18s
800:	learn: 0.9121286	total: 5m 47s	remaining: 2m 53s
1000:	learn: 0.9158143	total: 7m 19s	remaining: 1m 27s
1199:	learn: 0.9195857	total: 8m 46s	remaining: 0us
guardado


In [ ]:
from google.colab import files
files.download("catboost_treated.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>